# util

> Collection of helper functions to facilicate plotting

In [ ]:
#| default_exp util

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

from fastcore.test import *
from fastcore.docments import docments

In [ ]:
#| export
def procrustes(x:str, # input string
               appropriate_length:int=50, # desired length
               pad_with:str=" ", # character to pad with
               side:str="right" # which side to pad on ("left", "right")
              )->str: # string with desired length
    "A function to regulate string length."
    if len(x) > appropriate_length:
        return x[:appropriate_length]
    if len(x) < appropriate_length:
        to_pad = appropriate_length - len(x)
        pad = "".join([pad_with] * to_pad)
        if side == "right":
            x = x + pad
        elif side == "left":
            x = pad + x
        else:
            print("Invalid side argument; returning string as-is.")
    return x

We are primarily going to be working with non-model species, so the gene names will always be of the form

`XLOC_123456 | emapper-name-or-description-if-we're-lucky`

or something similar. This means that we could have extreme variation in the actual length of a gene "name"; this will make it very hard to put gene names on axes as it will distort figure sizes. I wrote a function to either trim or pad strings; even though axis labels are not in monospace fonts, it is much easier to visually reconcile strings with lengths in the same order of magnitude.

In [ ]:
too_short = 'Niko'
just_right = 'Theseus'
too_tall = 'The Mountain'

In [ ]:
assert procrustes(just_right, appropriate_length=7) == 'Theseus'
assert procrustes(too_short, appropriate_length=7) == 'Niko   '
assert procrustes(too_tall, appropriate_length=7) == 'The Mou'

In [ ]:
#| hide
import pandas as pd
import anndata as ad

<a id='grouped_obs_mean'></a>

In [ ]:
#| export
def grouped_obs_mean(adata:ad.AnnData, # AnnData object to analyse
                        group_key:str, # `.obs` category to group by
                        layer:str=None # layer to use. If none, use `.X`
                    )->pd.DataFrame: # a groups$\times$genes dataframe with the average expression
    "Helper function to calculate average expression per group in an `AnnData` object."
    if layer is not None:
        getX = lambda x: x.layers[layer]
    else:
        getX = lambda x: x.X

    grouped = adata.obs.groupby(group_key)
    out = pd.DataFrame(
        np.zeros((adata.shape[1], len(grouped)), dtype=np.float64),
        columns=list(grouped.groups.keys()),
        index=adata.var_names,
    )

    for group, idx in grouped.indices.items():
        X = getX(adata[idx])
        out[group] = np.ravel(X.mean(axis=0, dtype=np.float64))
    return out

Many tasks in single-cell analysis require us to know the average expression of a gene in a certain group of cells. While `scanpy` _does_ perform that task behind the scenes for, e.g. dotplots, this is not functionality that is exposed to the users. This is an implementation based on [ivirshup's answer](https://github.com/theislab/scanpy/issues/181#issuecomment-534867254) to a scanpy issue.

In [ ]:
import scanpy as sc
import numpy as np

adata = sc.read_h5ad('../example_data/hydra.h5ad')

cluster_means = grouped_obs_mean(adata, group_key='Cluster')

If $G$ is the number of genes and $C$ the number of unique clusters in the `group_key`, the returned array should have the shape $G \times C$:

In [ ]:
no_genes = adata.shape[1]
no_clusters = len(np.unique(adata.obs['Cluster']))
assert cluster_means.shape == (no_genes, no_clusters)

Additionally, each column of the array should contain the average detected expression for cells in that cluster:

In [ ]:
belong_to_ecEp_SC2 = adata.obs['Cluster'] == 'ecEp_SC2'
ecEp_SC2_average = np.mean(adata[belong_to_ecEp_SC2].X, axis=0)
ecEp_SC2_average = np.array(ecEp_SC2_average)[0]

assert all(np.isclose(cluster_means['ecEp_SC2'], ecEp_SC2_average))

In [ ]:
#| export
def grouped_obs_present(adata:ad.AnnData, # AnnData object to analyse
                        group_key:str, # `.obs` category to group by
                        layer:str=None # layer to use. If none, use `.X`
                       )->pd.DataFrame: # a groups$\times$genes dataframe with the number of expressing cells
    "Helper function to calculate how many cells express each gene per group in an `AnnData` object."
    if layer is not None:
        getX = lambda x: x.layers[layer]
    else:
        getX = lambda x: x.X

    grouped = adata.obs.groupby(group_key)
    out = pd.DataFrame(
        np.zeros((adata.shape[1], len(grouped)), dtype=np.float64),
        columns=list(grouped.groups.keys()),
        index=adata.var_names,
    )

    for group, idx in grouped.indices.items():
        X = getX(adata[idx])
        out[group] = np.ravel((X > 0).sum(axis=0, dtype=np.float64))
    return out

Another critical value to know when making dotplots is the fraction of cells expressing a gene in a certain cluster. Again, `scanpy` performs that task without exposing it to the users. Similar to [`grouped_obs_mean`](#grouped_obs_mean) this is an implementation based on [ivirshup's answer](https://github.com/theislab/scanpy/issues/181#issuecomment-534867254) to a scanpy issue. Here we calculate the sum of cells expressing a gene, a table we can use to calculate the fraction later.

In [ ]:
num_expressing = grouped_obs_present(adata, group_key='Cluster')

If $G$ is the number of genes and $C$ the number of unique clusters in the `group_key`, the returned array should have the shape $G \times C$:

In [ ]:
assert num_expressing.shape == (no_genes, no_clusters)

Additionally, each column of the array should contain the percentage of cells expressing each gene in that cluster:

In [ ]:
belong_to_ecEp_SC2 = adata.obs['Cluster'] == 'ecEp_SC2'
ecEp_SC2_expr = np.sum(adata[belong_to_ecEp_SC2].X>0, axis=0)
ecEp_SC2_expr = np.array(ecEp_SC2_expr)[0]

assert all(num_expressing['ecEp_SC2'] == ecEp_SC2_expr)

In [ ]:
#| export
def grouped_obs_percent(adata:ad.AnnData, # AnnData object to analyse
                        group_key:str, # `.obs` category to group by
                        layer:str=None # layer to use. If none, use `.X`
                       )->pd.DataFrame: # a groups$\times$genes dataframe with the number of expressing cells
    "Helper function to calculate how many cells express each gene per group in an `AnnData` object."
    num_expressing = grouped_obs_present(adata, group_key, layer=layer)
    no_cells_per_cluster = adata.obs[group_key].value_counts()
    return num_expressing / no_cells_per_cluster

Calculating the fraction of cells is of course very straightforward once we have counted the number of cells that express the gene as well as the total number of cells in a cluster.

In [ ]:
frac_expressing = grouped_obs_percent(adata, group_key='Cluster')
# use the counts and number of cells we calculated before
frac_ecEp_SC2 = ecEp_SC2_expr / np.sum(belong_to_ecEp_SC2)

assert all(np.isclose(frac_expressing['ecEp_SC2'], frac_ecEp_SC2))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()